In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# Upstage 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [4]:
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때 
index_name = 'tax-upstage-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

/Users/hs/.pyenv/versions/inflearn-llm-application/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [6]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()


In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/hs/.pyenv/versions/inflearn-llm-application/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [9]:
ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 450만원입니다. (산출 과정: 5,000만원 - 1,400만원 = 3,600만원, 3,600만원 * 15% + 84만원 = 594만원)'}